#### Imports

In [1]:
from dredFISH.Analysis.TissueGraph import *
from dredFISH.Visualization.Viz import *

from MERFISH_Objects.FISHData import *

from sklearn.preprocessing import normalize

import matplotlib.pyplot as plt 

#### Load data

#### Build layers 1-2: cells and zones

In [2]:
TMG = TissueMultiGraph('TMG_dev7.pkl')
TMG.Layers=TMG.Layers[0:2]

#### Build layers 3-4: communities and regions 

In [3]:
TMG.create_communities_and_region_layers(min_neigh_size = 10)


/home/rwollman/miniconda3/envs/dredFISH/lib/python3.9/site-packages/scipy/spatial/distance.py:1290: RuntimeWarning: invalid value encountered in true_divide
  p = p / np.sum(p, axis=axis, keepdims=True)
/home/rwollman/miniconda3/envs/dredFISH/lib/python3.9/site-packages/scipy/spatial/distance.py:1291: RuntimeWarning: invalid value encountered in true_divide
  q = q / np.sum(q, axis=axis, keepdims=True)
/home/rwollman/MyProjects/MaximallyInformativeBiocartography/dredFISH/dredFISH/Analysis/TissueGraph.py:883: RuntimeWarning: Mean of empty slice
  Smoothed[j,i+1] = np.nanmean(VecToSmooth[ix])


Calling initial optimization
 
 Func-count     x          f(x)          Procedure
    1        11.5208     -1.28394        initial
    2        18.5792     -1.36404        golden
    3        22.9416     -1.36269        golden
    4        20.6089     -1.40418        parabolic
    5        20.7424      -1.3733        parabolic
    6        19.6793      -1.3406        parabolic
    7        20.2538     -1.37771        golden
    8        20.4909     -1.38672        parabolic
    9        20.6599     -1.33532        golden
   10        20.5638     -1.33865        golden
   11        20.6283     -1.34424        golden
   12        20.5916      -1.3136        golden
   13        20.6163     -1.36303        golden
   14        20.6023     -1.34367        golden
   15        20.6122     -1.30492        golden

Optimization terminated successfully;
The returned value satisfies the termination criteria
(using xtol =  0.01 )
Number of types: 130 initial entropy: 1.4041761493535878 number of eva

In [4]:
TMG.save('TMG_dev8.pkl')